In [1]:
import ee
from geoEpic.gee import CompositeCollection

In [2]:
# conda create --name geo_epic python=3.11.9
# conda activate geo_epic
# pip install git+https://github.com/smarsGroup/geo-epic.git

In [4]:
col =  CompositeCollection('./HLS.yml')

In [12]:
#Extract hls the time series of a location
df = col.extract([[-98.114, 41.855]])
df

,Date,nir,red,green,blue,swir1,swir2,ndvi
0,2022-05-18,0.328,0.093,0.091,0.050,0.289,0.169,0.560
1,2022-05-22,0.365,0.108,0.123,0.114,0.319,0.179,0.544
2,2022-05-27,0.289,0.101,0.119,0.115,0.250,0.147,0.482
3,2022-06-02,0.353,0.053,0.068,0.034,0.206,0.099,0.739
4,2022-06-07,0.357,0.065,0.077,0.049,0.243,0.114,0.693
5,2022-06-11,0.244,0.067,0.069,0.034,0.198,0.109,0.569
6,2022-06-12,0.288,0.079,0.076,0.053,0.226,0.121,0.567
7,2022-06-16,0.311,0.088,0.106,0.097,0.273,0.144,0.558
8,2022-06-18,0.336,0.076,0.083,0.044,0.265,0.140,0.630
9,2022-06-19,0.346,0.070,0.081,0.040,0.265,0.138,0.662


In [ ]:
hls_ee = col.merged()
# hls_ee can be used as a normal earth engine collection.
hls_ee


In [53]:
# Below is an eaxmple of calculating and exporting ndvi.
# Filter collection to a rectangle in Nebraska
nebraska = ee.Geometry.Rectangle([-99.5, 41.0, -99.0, 41.5])
hls_ee_filtered =  hls_ee.filterBounds(nebraska)
   
# Function to calculate NDVI
def calc_NDVI(image):
    ndvi = image.normalizedDifference(['nir', 'red']).rename('NDVI')
    return ndvi

# Map the NDVI function over the collection
NDVI = hls_ee_filtered.map(calc_NDVI)

In [54]:
# clip NDVI and export to google drive
clippedNDVI = NDVI.mean().clip(nebraska)

task = ee.batch.Export.image.toDrive(
    image=clippedNDVI,
    description='Nebraska_NDVI_2020',
    scale=30,
    region=nebraska)
task.start()

In [2]:
#check status to see if export is completed
task.status()